In [ ]:
!pip install opendatasets

In [6]:
import opendatasets as od

In [3]:
#Baixando dataset mtbi do kaggle database
od.download("https://www.kaggle.com/datasets/datasnaek/mbti-type")

100%|██████████| 24.4M/24.4M [00:02<00:00, 9.90MB/s]


In [13]:
import pandas as pd
import numpy as np

In [7]:
#Importando dados para um dataframe
df = pd.read_csv('mbti-type/mbti_1.csv')
df.head()

,type,posts
0,INFJ,'http://www.youtube.com/watch?v=qsXHcwe3krw|||...
1,ENTP,'I'm finding the lack of me in these posts ver...
2,INTP,'Good one _____ https://www.youtube.com/wat...
3,INTJ,"'Dear INTP, I enjoyed our conversation the o..."
4,ENTJ,'You're fired.|||That's another silly misconce...


In [ ]:
#Imprimindo postos do primeiro usuário
df.iloc[0,1]

Nota-se que existem muitos links dentro dos posts, então há a necessidade de realizar tratamento dos dados, como a remoção de links, emojis, menções hashtags, pontuação, palavras com caixa alta, além de manter cada post em uma coluna separada e ainda, remover stopwords e manter somente o radical das palavras (stemming words).

In [25]:
#Importando libs necessárias
import re
import os
import emoji
import spacy
import string
from nltk.stem.porter import PorterStemmer
from sklearn.feature_extraction.text import ENGLISH_STOP_WORDS

In [37]:
#Criando função de tratamento
def tratamento_de_texto(texto, remover_stop_words = True, fazer_stemming = True, remover_mencoes_hashtags = True,
                        remover_links = True, remover_emojis = True):

    # Remoção de Emojis
    if remover_emojis == True:
        emoji_pattern = re.compile("[" "\U0001F1E0-\U0001F6FF" "]+", flags=re.UNICODE)
        texto = emoji_pattern.sub(r"", texto)
        texto = "".join([x for x in texto if x not in emoji.EMOJI_DATA])

    if remover_mencoes_hashtags:
        texto = re.sub(r"@(\w+)", " ", texto)
        texto = re.sub(r"#(\w+)", " ", texto)

    if remover_links:
        texto = re.sub(r'http\S+', '', texto)

    #Remoção de caracteres diversos
    texto = re.sub(r"[^\x00-\x7F]+", " ", texto)
    pontuacoes_e_numeros = re.compile('[' + re.escape(string.punctuation) + '0-9\\r\\t\\n]')
    nopunct = pontuacoes_e_numeros.sub(" ", texto.lower())
    palavras = (''.join(nopunct)).split()

    if(remover_stop_words):
        palavras = [p for p in palavras if p not in ENGLISH_STOP_WORDS]
        palavras = [p for p in palavras if len(p) > 2]  # remove a,an,of etc.

    if(fazer_stemming):
        stemmer = PorterStemmer()
        palavras = [stemmer.stem(p) for p in palavras]

    return list(palavras)

In [35]:
linhas, colunas = df.shape

In [41]:
#processando os textos
df['token'] = [tratamento_de_texto(df.iloc[linha][1]) for linha in range(linhas)]

In [43]:
#observando resultado
df.head()

,type,posts,token
0,INFJ,'http://www.youtube.com/watch?v=qsXHcwe3krw|||...,"[intj, moment, sportscent, play, prank, life, ..."
1,ENTP,'I'm finding the lack of me in these posts ver...,"[find, lack, post, alarm, sex, bore, posit, ex..."
2,INTP,'Good one _____ https://www.youtube.com/wat...,"[good, cours, say, know, bless, curs, doe, abs..."
3,INTJ,"'Dear INTP, I enjoyed our conversation the o...","[dear, intp, enjoy, convers, day, esoter, gab,..."
4,ENTJ,'You're fired.|||That's another silly misconce...,"[fire, silli, misconcept, approach, logic, go,..."


In [46]:
#salvando dados tratados
df.to_csv('mbti-type/mbti_tratado.csv', header = True)